In [1]:
# Load Modulues
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
import datajoint as dj
from datetime import datetime
import spikeinterface as si
from spikeinterface import widgets, exporters, postprocessing, qualitymetrics, sorters
from workflow.pipeline import *
from workflow.utils.paths import (
    get_ephys_root_data_dir,
    get_raw_root_data_dir,
    get_processed_root_data_dir,
)

import numpy as np
from matplotlib import pyplot as plt

ValueError: not enough values to unpack (expected at least 2, got 1)

In [ ]:
# Parameter Dictionaries
Batches = ["Batch 1" , "Batch 2" , "Batch 3"]
Drugs = ["4-AP" , "No Drug" , "Bicuculline" , "Tetrodotoxin"]
Organoids = {
    "Batch 1":["O09" , "O10" , "O11" , "O12"],
    "Batch 2":["O13" , "O14" , "O15" , "O16"],
    "Batch 3":["O17" , "O18" , "O19" , "O20"]
    }


In [ ]:
# Random functions

def get_dir_info(clustering_output_dir):
    # take clustering output dir and extract batch , drug_name , organoid , start_time , end_time

    # find organoid , start time , end time
    query = ephys.ClusteringTask() & f"clustering_output_dir = '{clustering_output_dir}'"
    organoid , experiment , start_time , end_time = query.fetch1("organoid_id" , "experiment_start_time" , "start_time" , "end_time")

    # find batch
    if organoid in Organoids["Batch 1"]:
        batch = "Batch 1"
    elif organoid in Organoids["Batch 2"]:
        batch = "Batch 2"
    elif organoid in Organoids["Batch 3"]:
        batch = "Batch 3"
    
    # find drug name
    drug = (culture.Experiment() & f"organoid_id = '{organoid}' AND experiment_start_time = '{experiment}'").fetch1("drug_name")

    info = {
        "batch":batch,
        "drug":drug,
        "organoid":organoid,
        "start_time":start_time,
        "end_time":end_time
        }
    
    return info

In [ ]:
# Get Data functions
class data:

    def get_tasks(global_key):
        # get clustering tasks which will contain data to be used (time segments and organoids)
        
        # global key = list of dictionaries of wanted Clustering Tasks - [{AND conditions} OR {AND conditions} OR ...]
        #   --> ex: [{"organoid_id":"O09"} , {"organoid_id":"O10" , "start_time":datetime(2023, 5, 18, 12, 25)}]
        #           = all tasks with organoid O09 or tasks with organoid O10 and start time 5/18/2023 , 12:25
        # key options: organoid_id , experiment_start_time , insertion_number , start_time , end_time , paramset_idx , clustering_output_dir

        # Get string key based on global key
        str_key = "" # initialize final string key

        for idx_OR , key in enumerate(global_key): # loop through global key (OR statements)
            
            str_OR = "" # intialize string for keys

            for idx_AND , (param , value) in enumerate(key.items()): # loop through individual keys (AND statements)

                str_AND = f"{param} = '{value}'" # create AND conditions

                if idx_AND == len(key)-1: # if the param,value pair isn't the last one (account for 0)
                    str_OR += str_AND # add onto OR condition without AND statement
                else:
                    str_OR += str_AND + " AND " # add onto OR condition with AND statement
                    
            if idx_OR == len(global_key)-1: # same logic as AND statements
                str_key += str_OR
            else:
                str_key += str_OR + " OR "

        # Extract wanted tasks
        tasks = ephys.ClusteringTask() & str_key
        
        return tasks

    def get_data(Tasks , global_query , Values):
        # get data values based on clustering task

        # Tasks = query of clustering tasks generated by get_tasks
        # global_query = query where to access the data
        # Values = wanted data values (ex: amplitude, firing rate, electrode, waveform)

        data = {}

        for task in Tasks: # loop through wanted values

            # get task info
            clustering_output_dir = task["clustering_output_dir"]
            task_info = get_dir_info(clustering_output_dir=clustering_output_dir)
            task_title = "/".join(map(str, task_info.values()))

            # get task data query
            task_query = global_query & f"organoid_id='{task_info['organoid']}' AND start_time='{task_info['start_time']}' AND end_time='{task_info['end_time']}'"

            if len(task_query) == 0:

                data[task_title] = "no data"

            else:
                
                data[task_title] = {}
                for value in Values: # loop through tasks
                    
                    # extract value data
                    value_data = task_query.fetch(value)

                    # put into dictionary
                    data[task_title][value] = value_data

        return data

In [ ]:
# Plot Data functions

class plot:

    def get_bar(bar , errorbar , points , bargap=.2 , groupgap=.2):
        # make bar plot

        # bar = height of bars (2D numpy , each 1D are the values for a single group)
        # errorbar = height of errorbar (2D numpy , 0 for no bar)
        # points = any points to be plotted (2D numpy , each 1D numpy is for a group , it's a list of tuples each representing points for a bar)
        # bargap = gap between bars (center to center)
        # groupgap = gap between groups (center to center)

        # Get X values (along x axis)
        numgroup , numbar = bar.shape
        xvalues = np.tile(np.arange(numbar) , (numgroup,1)) 
        groupoffset = (np.arange(numgroup)[:, np.newaxis]) * (np.ones((1, numbar)) * (groupgap + bargap*numbar))
        xvalues += groupoffset

        # Initialize Axis lists
        bar_ax = []
        error_ax = []
        points_ax = []      

        for i in range(numgroup):
            
            # Get group values
            x_i = xvalues[i]
            bar_i = bar[i]
            error_i = errorbar[i]
            points_i = points[i]

            # Plot
            bar_ax.append(plt.bar(x_i , bar_i , bargap , edgecolor='k'))
            error_ax.append(plt.errorbar(x_i , bar_i , yerr=error_i , fmt='o' , color='k' , capsize=4 , markersize=0))
            points_ax.append([])
            for ii in range(numbar):
        
                x_ii = x_i[ii]
                points_ii = points_i[ii]

                points_ax[i].append(plt.scatter([x_ii]*len(points_ii) + (np.random.random(len(points_ii)))*(bargap/2)-(bargap/4) , points_ii))

        return bar_ax , error_ax , points_ax

In [ ]:
# Get Data Example

global_key = [{"organoid_id":"O09" , "experiment_start_time":datetime(2023, 5, 18, 12, 25)}]
Tasks = data.get_tasks(global_key=global_key)

global_query = ephys.QualityMetrics.Cluster()
Values = ["firing_rate" , "snr"]

data = data.get_data(Tasks=Tasks , global_query=global_query , Values=Values)